In [1]:
import pandas as pd
import os
import numpy as np

import plotly.express as px

from data.constants import DATA_FOLDER

from power import power_uncond
from tabulate import  tabulate

In [2]:
comm_panel_in = os.path.join(DATA_FOLDER, "final", "comm_panel.parquet")
tract_panel_in = os.path.join(DATA_FOLDER, "final", "tract_panel.parquet")

In [3]:
comm_panel = pd.read_parquet(comm_panel_in)
comm_panel = comm_panel.query('~is_weekend')
tract_panel = pd.read_parquet(tract_panel_in)
tract_panel = tract_panel.query('~is_weekend')

In [4]:
trt_panel = tract_panel.query("(DNC==1)")

In [5]:
power_stats_base = power_uncond(trt_panel, ['transit','UCMP'], 'rides', 0)[['transit','UCMP','mean','std','mde','mds']]
power_stats_base['mds'] /= 5e4
plot_data = power_stats_base.round(1).set_index(['UCMP','transit']).sort_index()

In [6]:
plot_data.index.set_names(['Serves DNC', 'Transit'], inplace=True)
plot_data.columns.set_names('Daily Rides', inplace=True)
plot_data['Mean (SD)'] = plot_data['mean'].astype(str) + plot_data['std'].apply(" ({})".format)
plot_data.drop(columns=['mean','std'], inplace=True)
plot_data.rename(columns={'mde': 'Minimum Detectable Effect',
                          'mds': 'MDE Per Attendee'}, inplace=True)
plot_data = plot_data.iloc[:, [2,0,1]]
plot_data

Daily Rides               Mean (SD)  Minimum Detectable Effect  \
Serves DNC Transit                                               
0          bike        72.8 (181.6)                       12.4   
           train    2651.2 (4084.7)                      584.0   
           uber       187.5 (685.8)                       31.1   
1          bike       177.0 (237.1)                       55.4   
           train    2492.9 (2102.1)                      787.0   
           uber      925.3 (1981.5)                      438.9   

Daily Rides         MDE Per Attendee  
Serves DNC Transit                    
0          bike                  0.4  
           train                 4.5  
           uber                  2.4  
1          bike                  0.2  
           train                 0.9  
           uber                  1.4

In [7]:
with open("../../../reports/replication/mde.md","w") as f:
    tbl = tabulate(plot_data.reset_index(), 
                   headers=plot_data.reset_index().columns, 
                   showindex=False,
                   tablefmt='github')
    f.write(tbl)
    print(tbl)
!cp ../../../reports/replication/mde.md ../../../../eric-mc2-cv/static/uploads/

|   Serves DNC | Transit   | Mean (SD)       |   Minimum Detectable Effect |   MDE Per Attendee |
|--------------|-----------|-----------------|-----------------------------|--------------------|
|            0 | bike      | 72.8 (181.6)    |                        12.4 |                0.4 |
|            0 | train     | 2651.2 (4084.7) |                       584   |                4.5 |
|            0 | uber      | 187.5 (685.8)   |                        31.1 |                2.4 |
|            1 | bike      | 177.0 (237.1)   |                        55.4 |                0.2 |
|            1 | train     | 2492.9 (2102.1) |                       787   |                0.9 |
|            1 | uber      | 925.3 (1981.5)  |                       438.9 |                1.4 |
